In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from scipy import sparse
import re

In [ ]:
path = '../CMB/0430/'

In [ ]:
beh = pd.read_csv(path+'train_beh.csv')
tag = pd.read_csv(path+'train_tag.csv')
trd = pd.read_csv(path+'train_trd.csv')

In [ ]:
beh_a = pd.read_csv(path+'test_beh_a.csv')
tag_a = pd.read_csv(path+'test_tag_a.csv')
trd_a = pd.read_csv(path+'test_trd_a.csv')

In [ ]:
beh_b = pd.read_csv(path+'test_beh_b.csv')
tag_b = pd.read_csv(path+'test_tag_b.csv')
trd_b = pd.read_csv(path+'test_trd_b.csv')

In [ ]:
train = tag
test = tag_a
test['flag']=-1
data=pd.concat([train,test])
data = data.fillna('-1')

In [ ]:
train_t = trd
test_t = trd_a
test_t['flag']=-1
data_t=pd.concat([train_t,test_t])
data_t=data_t.fillna(0)

In [ ]:
train_b = beh
test_b = beh_a
test_b['flag']=-1
data_b = pd.concat([train_b, test_b])

In [ ]:
def function(string):
    return bool(re.match('\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$', string))

data_t['trx_tm'] = data_t['trx_tm'].apply(lambda x: '2019-07-01 00:00:00' if not function(str(x)) else x)
data_b['page_tm'] = data_b['page_tm'].apply(lambda x: '2019-07-01 00:00:00' if not function(str(x)) else x)
data_t['trx_day'] = data_t['trx_tm'].apply(lambda x:str(x).split()[0])
data_t['trx_hour'] = data_t['trx_tm'].apply(lambda x:str(x).split()[1].split(':')[0])
data_b['page_day'] = data_b['page_tm'].apply(lambda x:str(x).split()[0])
data_b['page_hour'] = data_b['page_tm'].apply(lambda x:str(x).split()[1].split(':')[0])

data_t['trx_tm'] = pd.to_datetime(data_t['trx_tm'], format ='%Y-%m-%d %H:%M:%S')
data_b['page_tm'] = pd.to_datetime(data_b['page_tm'], format ='%Y-%m-%d %H:%M:%S')

In [ ]:
#交易频率最高的支付方式 
data_t_sta = data_t.groupby(['id', 'Dat_Flg3_Cd'], as_index=False).count()
data_t_sta.sort_values('flag', ascending=False, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg3_Cd']]
data_t_sta.rename(columns={'Dat_Flg3_Cd':'Fre_Trans_Flg3'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Fre_Trans_Flg3'] = data['Fre_Trans_Flg3'].fillna('-1')

In [ ]:
#收入频率最高的支付方式 
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta.groupby(['id', 'Dat_Flg3_Cd'], as_index=False).count()
data_t_sta.sort_values('flag', ascending=False, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg3_Cd']]
data_t_sta.rename(columns={'Dat_Flg3_Cd':'Fre_Inc_Flg3'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Fre_Inc_Flg3'] = data['Fre_Inc_Flg3'].fillna('-1')

In [ ]:
#支出频率最高的支付方式 
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta.groupby(['id', 'Dat_Flg3_Cd'], as_index=False).count()
data_t_sta.sort_values('flag', ascending=False, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg3_Cd']]
data_t_sta.rename(columns={'Dat_Flg3_Cd':'Fre_Pay_Flg3'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Fre_Pay_Flg3'] = data['Fre_Pay_Flg3'].fillna('-1')

In [ ]:
#使用支付方式A进行交易次数
data_t_sta = data_t[data_t['Dat_Flg3_Cd']=='A']
data_t_sta = data_t_sta.groupby('id', as_index=False).count()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg3_Cd']]
data_t_sta.rename(columns={'Dat_Flg3_Cd':'Num_Trans_Flg3_A'}, inplace=True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Num_Trans_Flg3_A'] = data['Num_Trans_Flg3_A'].fillna(0)

In [ ]:
#使用支付方式B进行交易次数
data_t_sta = data_t[data_t['Dat_Flg3_Cd']=='B']
data_t_sta = data_t_sta.groupby('id', as_index=False).count()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg3_Cd']]
data_t_sta.rename(columns={'Dat_Flg3_Cd':'Num_Trans_Flg3_B'}, inplace=True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Num_Trans_Flg3_B'] = data['Num_Trans_Flg3_B'].fillna(0)

In [ ]:
#使用支付方式C进行交易次数
data_t_sta = data_t[data_t['Dat_Flg3_Cd']=='C']
data_t_sta = data_t_sta.groupby('id', as_index=False).count()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg3_Cd']]
data_t_sta.rename(columns={'Dat_Flg3_Cd':'Num_Trans_Flg3_C'}, inplace=True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Num_Trans_Flg3_C'] = data['Num_Trans_Flg3_C'].fillna(0)

In [ ]:
#使用支付方式A进行收入次数
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='A']
data_t_sta = data_t_sta.groupby('id', as_index=False).count()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg3_Cd']]
data_t_sta.rename(columns={'Dat_Flg3_Cd':'Num_Inc_Flg3_A'}, inplace=True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Num_Inc_Flg3_A'] = data['Num_Inc_Flg3_A'].fillna(0)

In [ ]:
#使用支付方式B进行收入次数
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='B']
data_t_sta = data_t_sta.groupby('id', as_index=False).count()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg3_Cd']]
data_t_sta.rename(columns={'Dat_Flg3_Cd':'Num_Inc_Flg3_B'}, inplace=True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Num_Inc_Flg3_B'] = data['Num_Inc_Flg3_B'].fillna(0)

In [ ]:
#使用支付方式C进行收入次数
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='C']
data_t_sta = data_t_sta.groupby('id', as_index=False).count()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg3_Cd']]
data_t_sta.rename(columns={'Dat_Flg3_Cd':'Num_Inc_Flg3_C'}, inplace=True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Num_Inc_Flg3_C'] = data['Num_Inc_Flg3_C'].fillna(0)

In [ ]:
#使用支付方式A进行支出次数
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='A']
data_t_sta = data_t_sta.groupby('id', as_index=False).count()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg3_Cd']]
data_t_sta.rename(columns={'Dat_Flg3_Cd':'Num_Pay_Flg3_A'}, inplace=True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Num_Pay_Flg3_A'] = data['Num_Pay_Flg3_A'].fillna(0)

In [ ]:
#使用支付方式B进行支出次数
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='B']
data_t_sta = data_t_sta.groupby('id', as_index=False).count()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg3_Cd']]
data_t_sta.rename(columns={'Dat_Flg3_Cd':'Num_Pay_Flg3_B'}, inplace=True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Num_Pay_Flg3_B'] = data['Num_Pay_Flg3_B'].fillna(0)

In [ ]:
#使用支付方式C进行支出次数
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='C']
data_t_sta = data_t_sta.groupby('id',as_index=False).count()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg3_Cd']]
data_t_sta.rename(columns={'Dat_Flg3_Cd':'Num_Pay_Flg3_C'}, inplace=True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Num_Pay_Flg3_C'] = data['Num_Pay_Flg3_C'].fillna(0)

In [ ]:
#第一次使用支付方式A进行交易的收支方向/一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg3_Cd']=='A']
data_t_sta.sort_values('trx_tm', ascending=True, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg1_Cd', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Dat_Flg1_Cd':'First_Trans_Flg1_A', 'Trx_Cod1_Cd':'First_Trans_Cod1_A',
                           'Trx_Cod2_Cd':'First_Trans_Cod2_A', 'trx_tm':'First_Trans_Time_A',
                           'cny_trx_amt':'First_Trans_Amt_A'}, inplace=True)
data_t_sta['First_Trans_Time_A'] = pd.to_numeric(data_t_sta['First_Trans_Time_A'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['First_Trans_Flg1_A'] = data['First_Trans_Flg1_A'].fillna('-1')
data['First_Trans_Cod1_A'] = data['First_Trans_Cod1_A'].fillna(-1)
data['First_Trans_Cod2_A'] = data['First_Trans_Cod2_A'].fillna(-1)
data['First_Trans_Time_A'] = data['First_Trans_Time_A'].fillna(-1)
data['First_Trans_Amt_A'] = data['First_Trans_Amt_A'].fillna(0)

In [ ]:
#第一次使用支付方式B进行交易的收支方向/一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg3_Cd']=='B']
data_t_sta.sort_values('trx_tm', ascending=True, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg1_Cd', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Dat_Flg1_Cd':'First_Trans_Flg1_B', 'Trx_Cod1_Cd':'First_Trans_Cod1_B',
                           'Trx_Cod2_Cd':'First_Trans_Cod2_B', 'trx_tm':'First_Trans_Time_B',
                           'cny_trx_amt':'First_Trans_Amt_B'}, inplace=True)
data_t_sta['First_Trans_Time_B'] = pd.to_numeric(data_t_sta['First_Trans_Time_B'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['First_Trans_Flg1_B'] = data['First_Trans_Flg1_B'].fillna('-1')
data['First_Trans_Cod1_B'] = data['First_Trans_Cod1_B'].fillna(-1)
data['First_Trans_Cod2_B'] = data['First_Trans_Cod2_B'].fillna(-1)
data['First_Trans_Time_B'] = data['First_Trans_Time_B'].fillna(-1)
data['First_Trans_Amt_B'] = data['First_Trans_Amt_B'].fillna(0)

In [ ]:
#第一次使用支付方式C进行交易的收支方向/一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg3_Cd']=='C']
data_t_sta.sort_values('trx_tm', ascending=True, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg1_Cd', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Dat_Flg1_Cd':'First_Trans_Flg1_C', 'Trx_Cod1_Cd':'First_Trans_Cod1_C',
                           'Trx_Cod2_Cd':'First_Trans_Cod2_C', 'trx_tm':'First_Trans_Time_C',
                           'cny_trx_amt':'First_Trans_Amt_C'}, inplace=True)
data_t_sta['First_Trans_Time_C'] = pd.to_numeric(data_t_sta['First_Trans_Time_C'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['First_Trans_Flg1_C'] = data['First_Trans_Flg1_C'].fillna('-1')
data['First_Trans_Cod1_C'] = data['First_Trans_Cod1_C'].fillna(-1)
data['First_Trans_Cod2_C'] = data['First_Trans_Cod2_C'].fillna(-1)
data['First_Trans_Time_C'] = data['First_Trans_Time_C'].fillna(-1)
data['First_Trans_Amt_C'] = data['First_Trans_Amt_C'].fillna(0)

In [ ]:
#第一次使用支付方式A进行收入的一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='A']
data_t_sta.sort_values('trx_tm', ascending=True, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Trx_Cod1_Cd':'First_Inc_Cod1_A', 'Trx_Cod2_Cd':'First_Inc_Cod2_A',
                           'trx_tm':'First_Inc_Time_A', 'cny_trx_amt':'First_Inc_Amt_A'}, inplace=True)
data_t_sta['First_Inc_Time_A'] = pd.to_numeric(data_t_sta['First_Inc_Time_A'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['First_Inc_Cod1_A'] = data['First_Inc_Cod1_A'].fillna(-1)
data['First_Inc_Cod2_A'] = data['First_Inc_Cod2_A'].fillna(-1)
data['First_Inc_Time_A'] = data['First_Inc_Time_A'].fillna(-1)
data['First_Inc_Amt_A'] = data['First_Inc_Amt_A'].fillna(0)

In [ ]:
#第一次使用支付方式B进行收入的一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='B']
data_t_sta.sort_values('trx_tm', ascending=True, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Trx_Cod1_Cd':'First_Inc_Cod1_B', 'Trx_Cod2_Cd':'First_Inc_Cod2_B',
                           'trx_tm':'First_Inc_Time_B', 'cny_trx_amt':'First_Inc_Amt_B'}, inplace=True)
data_t_sta['First_Inc_Time_B'] = pd.to_numeric(data_t_sta['First_Inc_Time_B'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['First_Inc_Cod1_B'] = data['First_Inc_Cod1_B'].fillna(-1)
data['First_Inc_Cod2_B'] = data['First_Inc_Cod2_B'].fillna(-1)
data['First_Inc_Time_B'] = data['First_Inc_Time_B'].fillna(-1)
data['First_Inc_Amt_B'] = data['First_Inc_Amt_B'].fillna(0)

In [ ]:
#第一次使用支付方式C进行收入的一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='C']
data_t_sta.sort_values('trx_tm', ascending=True, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Trx_Cod1_Cd':'First_Inc_Cod1_C', 'Trx_Cod2_Cd':'First_Inc_Cod2_C',
                           'trx_tm':'First_Inc_Time_C', 'cny_trx_amt':'First_Inc_Amt_C'}, inplace=True)
data_t_sta['First_Inc_Time_C'] = pd.to_numeric(data_t_sta['First_Inc_Time_C'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['First_Inc_Cod1_C'] = data['First_Inc_Cod1_C'].fillna(-1)
data['First_Inc_Cod2_C'] = data['First_Inc_Cod2_C'].fillna(-1)
data['First_Inc_Time_C'] = data['First_Inc_Time_C'].fillna(-1)
data['First_Inc_Amt_C'] = data['First_Inc_Amt_C'].fillna(0)

In [ ]:
#第一次使用支付方式A进行支出的一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='A']
data_t_sta.sort_values('trx_tm', ascending=True, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Trx_Cod1_Cd':'First_Pay_Cod1_A', 'Trx_Cod2_Cd':'First_Pay_Cod2_A',
                           'trx_tm':'First_Pay_Time_A', 'cny_trx_amt':'First_Pay_Amt_A'}, inplace=True)
data_t_sta['First_Pay_Time_A'] = pd.to_numeric(data_t_sta['First_Pay_Time_A'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['First_Pay_Cod1_A'] = data['First_Pay_Cod1_A'].fillna(-1)
data['First_Pay_Cod2_A'] = data['First_Pay_Cod2_A'].fillna(-1)
data['First_Pay_Time_A'] = data['First_Pay_Time_A'].fillna(-1)
data['First_Pay_Amt_A'] = data['First_Pay_Amt_A'].fillna(0)

In [ ]:
#第一次使用支付方式B进行支出的一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='B']
data_t_sta.sort_values('trx_tm', ascending=True, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Trx_Cod1_Cd':'First_Pay_Cod1_B', 'Trx_Cod2_Cd':'First_Pay_Cod2_B',
                           'trx_tm':'First_Pay_Time_B', 'cny_trx_amt':'First_Pay_Amt_B'}, inplace=True)
data_t_sta['First_Pay_Time_B'] = pd.to_numeric(data_t_sta['First_Pay_Time_B'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['First_Pay_Cod1_B'] = data['First_Pay_Cod1_B'].fillna(-1)
data['First_Pay_Cod2_B'] = data['First_Pay_Cod2_B'].fillna(-1)
data['First_Pay_Time_B'] = data['First_Pay_Time_B'].fillna(-1)
data['First_Pay_Amt_B'] = data['First_Pay_Amt_B'].fillna(0)

In [ ]:
#第一次使用支付方式C进行支出的一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='C']
data_t_sta.sort_values('trx_tm', ascending=True, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Trx_Cod1_Cd':'First_Pay_Cod1_C', 'Trx_Cod2_Cd':'First_Pay_Cod2_C',
                           'trx_tm':'First_Pay_Time_C', 'cny_trx_amt':'First_Pay_Amt_C'}, inplace=True)
data_t_sta['First_Pay_Time_C'] = pd.to_numeric(data_t_sta['First_Pay_Time_C'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['First_Pay_Cod1_C'] = data['First_Pay_Cod1_C'].fillna(-1)
data['First_Pay_Cod2_C'] = data['First_Pay_Cod2_C'].fillna(-1)
data['First_Pay_Time_C'] = data['First_Pay_Time_C'].fillna(-1)
data['First_Pay_Amt_C'] = data['First_Pay_Amt_C'].fillna(0)

In [ ]:
#最后一次使用支付方式A进行交易的收支方向/一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg3_Cd']=='A']
data_t_sta.sort_values('trx_tm', ascending=False, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg1_Cd', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Dat_Flg1_Cd':'Last_Trans_Flg1_A', 'Trx_Cod1_Cd':'Last_Trans_Cod1_A',
                           'Trx_Cod2_Cd':'Last_Trans_Cod2_A', 'trx_tm':'Last_Trans_Time_A',
                           'cny_trx_amt':'Last_Trans_Amt_A'}, inplace=True)
data_t_sta['Last_Trans_Time_A'] = pd.to_numeric(data_t_sta['Last_Trans_Time_A'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Last_Trans_Flg1_A'] = data['Last_Trans_Flg1_A'].fillna('-1')
data['Last_Trans_Cod1_A'] = data['Last_Trans_Cod1_A'].fillna(-1)
data['Last_Trans_Cod2_A'] = data['Last_Trans_Cod2_A'].fillna(-1)
data['Last_Trans_Time_A'] = data['Last_Trans_Time_A'].fillna(-1)
data['Last_Trans_Amt_A'] = data['Last_Trans_Amt_A'].fillna(0)

In [ ]:
#最后一次使用支付方式B进行交易的收支方向/一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg3_Cd']=='B']
data_t_sta.sort_values('trx_tm', ascending=False, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg1_Cd', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Dat_Flg1_Cd':'Last_Trans_Flg1_B', 'Trx_Cod1_Cd':'Last_Trans_Cod1_B',
                           'Trx_Cod2_Cd':'Last_Trans_Cod2_B', 'trx_tm':'Last_Trans_Time_B',
                           'cny_trx_amt':'Last_Trans_Amt_B'}, inplace=True)
data_t_sta['Last_Trans_Time_B'] = pd.to_numeric(data_t_sta['Last_Trans_Time_B'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Last_Trans_Flg1_B'] = data['Last_Trans_Flg1_B'].fillna('-1')
data['Last_Trans_Cod1_B'] = data['Last_Trans_Cod1_B'].fillna(-1)
data['Last_Trans_Cod2_B'] = data['Last_Trans_Cod2_B'].fillna(-1)
data['Last_Trans_Time_B'] = data['Last_Trans_Time_B'].fillna(-1)
data['Last_Trans_Amt_B'] = data['Last_Trans_Amt_B'].fillna(0)

In [ ]:
#最后一次使用支付方式C进行交易的收支方向/一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg3_Cd']=='C']
data_t_sta.sort_values('trx_tm', ascending=False, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg1_Cd', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Dat_Flg1_Cd':'Last_Trans_Flg1_C', 'Trx_Cod1_Cd':'Last_Trans_Cod1_C',
                           'Trx_Cod2_Cd':'Last_Trans_Cod2_C', 'trx_tm':'Last_Trans_Time_C',
                           'cny_trx_amt':'Last_Trans_Amt_C'}, inplace=True)
data_t_sta['Last_Trans_Time_C'] = pd.to_numeric(data_t_sta['Last_Trans_Time_C'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Last_Trans_Flg1_C'] = data['Last_Trans_Flg1_C'].fillna('-1')
data['Last_Trans_Cod1_C'] = data['Last_Trans_Cod1_C'].fillna(-1)
data['Last_Trans_Cod2_C'] = data['Last_Trans_Cod2_C'].fillna(-1)
data['Last_Trans_Time_C'] = data['Last_Trans_Time_C'].fillna(-1)
data['Last_Trans_Amt_C'] = data['Last_Trans_Amt_C'].fillna(0)

In [ ]:
#最后一次使用支付方式A进行收入的一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='A']
data_t_sta.sort_values('trx_tm', ascending=False, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Trx_Cod1_Cd':'Last_Inc_Cod1_A', 'Trx_Cod2_Cd':'Last_Inc_Cod2_A',
                           'trx_tm':'Last_Inc_Time_A', 'cny_trx_amt':'Last_Inc_Amt_A'}, inplace=True)
data_t_sta['Last_Inc_Time_A'] = pd.to_numeric(data_t_sta['Last_Inc_Time_A'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Last_Inc_Cod1_A'] = data['Last_Inc_Cod1_A'].fillna(-1)
data['Last_Inc_Cod2_A'] = data['Last_Inc_Cod2_A'].fillna(-1)
data['Last_Inc_Time_A'] = data['Last_Inc_Time_A'].fillna(-1)
data['Last_Inc_Amt_A'] = data['Last_Inc_Amt_A'].fillna(0)

In [ ]:
#最后一次使用支付方式B进行收入的一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='B']
data_t_sta.sort_values('trx_tm', ascending=False, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Trx_Cod1_Cd':'Last_Inc_Cod1_B', 'Trx_Cod2_Cd':'Last_Inc_Cod2_B',
                           'trx_tm':'Last_Inc_Time_B', 'cny_trx_amt':'Last_Inc_Amt_B'}, inplace=True)
data_t_sta['Last_Inc_Time_B'] = pd.to_numeric(data_t_sta['Last_Inc_Time_B'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Last_Inc_Cod1_B'] = data['Last_Inc_Cod1_B'].fillna(-1)
data['Last_Inc_Cod2_B'] = data['Last_Inc_Cod2_B'].fillna(-1)
data['Last_Inc_Time_B'] = data['Last_Inc_Time_B'].fillna(-1)
data['Last_Inc_Amt_B'] = data['Last_Inc_Amt_B'].fillna(0)

In [ ]:
#最后一次使用支付方式C进行收入的一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='C']
data_t_sta.sort_values('trx_tm', ascending=False, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Trx_Cod1_Cd':'Last_Inc_Cod1_C', 'Trx_Cod2_Cd':'Last_Inc_Cod2_C',
                           'trx_tm':'Last_Inc_Time_C', 'cny_trx_amt':'Last_Inc_Amt_C'}, inplace=True)
data_t_sta['Last_Inc_Time_C'] = pd.to_numeric(data_t_sta['Last_Inc_Time_C'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Last_Inc_Cod1_C'] = data['Last_Inc_Cod1_C'].fillna(-1)
data['Last_Inc_Cod2_C'] = data['Last_Inc_Cod2_C'].fillna(-1)
data['Last_Inc_Time_C'] = data['Last_Inc_Time_C'].fillna(-1)
data['Last_Inc_Amt_C'] = data['Last_Inc_Amt_C'].fillna(0)

In [ ]:
#最后一次使用支付方式A进行支出的一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='A']
data_t_sta.sort_values('trx_tm', ascending=False, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Trx_Cod1_Cd':'Last_Pay_Cod1_A', 'Trx_Cod2_Cd':'Last_Pay_Cod2_A',
                           'trx_tm':'Last_Pay_Time_A', 'cny_trx_amt':'Last_Pay_Amt_A'}, inplace=True)
data_t_sta['Last_Pay_Time_A'] = pd.to_numeric(data_t_sta['Last_Pay_Time_A'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Last_Pay_Cod1_A'] = data['Last_Pay_Cod1_A'].fillna(-1)
data['Last_Pay_Cod2_A'] = data['Last_Pay_Cod2_A'].fillna(-1)
data['Last_Pay_Time_A'] = data['Last_Pay_Time_A'].fillna(-1)
data['Last_Pay_Amt_A'] = data['Last_Pay_Amt_A'].fillna(0)

In [ ]:
#最后一次使用支付方式B进行支出的一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='B']
data_t_sta.sort_values('trx_tm', ascending=False, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Trx_Cod1_Cd':'Last_Pay_Cod1_B', 'Trx_Cod2_Cd':'Last_Pay_Cod2_B',
                           'trx_tm':'Last_Pay_Time_B', 'cny_trx_amt':'Last_Pay_Amt_B'}, inplace=True)
data_t_sta['Last_Pay_Time_B'] = pd.to_numeric(data_t_sta['Last_Pay_Time_B'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Last_Pay_Cod1_B'] = data['Last_Pay_Cod1_B'].fillna(-1)
data['Last_Pay_Cod2_B'] = data['Last_Pay_Cod2_B'].fillna(-1)
data['Last_Pay_Time_B'] = data['Last_Pay_Time_B'].fillna(-1)
data['Last_Pay_Amt_B'] = data['Last_Pay_Amt_B'].fillna(0)

In [ ]:
#最后一次使用支付方式C进行支出的一级分类代码/二级分类代码/时间/金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta[data_t_sta['Dat_Flg3_Cd']=='C']
data_t_sta.sort_values('trx_tm', ascending=False, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt']]
data_t_sta.rename(columns={'Trx_Cod1_Cd':'Last_Pay_Cod1_C', 'Trx_Cod2_Cd':'Last_Pay_Cod2_C',
                           'trx_tm':'Last_Pay_Time_C', 'cny_trx_amt':'Last_Pay_Amt_C'}, inplace=True)
data_t_sta['Last_Pay_Time_C'] = pd.to_numeric(data_t_sta['Last_Pay_Time_C'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Last_Pay_Cod1_C'] = data['Last_Pay_Cod1_C'].fillna(-1)
data['Last_Pay_Cod2_C'] = data['Last_Pay_Cod2_C'].fillna(-1)
data['Last_Pay_Time_C'] = data['Last_Pay_Time_C'].fillna(-1)
data['Last_Pay_Amt_C'] = data['Last_Pay_Amt_C'].fillna(0)

In [ ]:
#支付方式A在交易中占比
data_t_sta = data_t.groupby('id', as_index=False).count()
tmp = data_t[data_t['Dat_Flg3_Cd']=='A']
tmp = tmp.groupby('id', as_index=False)['Dat_Flg3_Cd'].count()
tmp.rename(columns={'Dat_Flg3_Cd':'Num_Trans_Flg3'}, inplace=True)
data_t_sta = pd.merge(data_t_sta, tmp, on='id', how='left')
data_t_sta['Num_Trans_Flg3'] = data_t_sta['Num_Trans_Flg3'].fillna(0)
data_t_sta['Prop_Trans_Flg3_A'] = data_t_sta.apply(lambda x:0 if x['Num_Trans_Flg3']==0 else x['Num_Trans_Flg3']/x['Dat_Flg3_Cd'], axis=1)
data_t_sta = data_t_sta.loc[:, ['id', 'Prop_Trans_Flg3_A']]
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Prop_Trans_Flg3_A'] = data['Prop_Trans_Flg3_A'].fillna(0)

In [ ]:
#支付方式B在交易中占比
data_t_sta = data_t.groupby('id', as_index=False).count()
tmp = data_t[data_t['Dat_Flg3_Cd']=='B']
tmp = tmp.groupby('id', as_index=False)['Dat_Flg3_Cd'].count()
tmp.rename(columns={'Dat_Flg3_Cd':'Num_Trans_Flg3'}, inplace=True)
data_t_sta = pd.merge(data_t_sta, tmp, on='id', how='left')
data_t_sta['Num_Trans_Flg3'] = data_t_sta['Num_Trans_Flg3'].fillna(0)
data_t_sta['Prop_Trans_Flg3_B'] = data_t_sta.apply(lambda x:x['Num_Trans_Flg3']/x['Dat_Flg3_Cd'], axis=1)
data_t_sta = data_t_sta.loc[:, ['id', 'Prop_Trans_Flg3_B']]
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Prop_Trans_Flg3_B'] = data['Prop_Trans_Flg3_B'].fillna(0)

In [ ]:
#支付方式C在交易中占比
data_t_sta = data_t.groupby('id', as_index=False).count()
tmp = data_t[data_t['Dat_Flg3_Cd']=='C']
tmp = tmp.groupby('id', as_index=False)['Dat_Flg3_Cd'].count()
tmp.rename(columns={'Dat_Flg3_Cd':'Num_Trans_Flg3'}, inplace=True)
data_t_sta = pd.merge(data_t_sta, tmp, on='id', how='left')
data_t_sta['Num_Trans_Flg3'] = data_t_sta['Num_Trans_Flg3'].fillna(0)
data_t_sta['Prop_Trans_Flg3_C'] = data_t_sta.apply(lambda x:x['Num_Trans_Flg3']/x['Dat_Flg3_Cd'], axis=1)
data_t_sta = data_t_sta.loc[:, ['id', 'Prop_Trans_Flg3_C']]
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Prop_Trans_Flg3_C'] = data['Prop_Trans_Flg3_C'].fillna(0)

In [ ]:
#支付方式A在收入中占比
data_t_C = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_C.groupby('id', as_index=False).count()
tmp = data_t_C[data_t_C['Dat_Flg3_Cd']=='A']
tmp = tmp.groupby('id', as_index=False)['Dat_Flg3_Cd'].count()
tmp.rename(columns={'Dat_Flg3_Cd':'Num_Inc_Flg3'}, inplace=True)
data_t_sta = pd.merge(data_t_sta, tmp, on='id', how='left')
data_t_sta['Num_Inc_Flg3'] = data_t_sta['Num_Inc_Flg3'].fillna(0)
data_t_sta['Prop_Inc_Flg3_A'] = data_t_sta.apply(lambda x:x['Num_Inc_Flg3']/x['Dat_Flg3_Cd'], axis=1)
data_t_sta = data_t_sta.loc[:, ['id', 'Prop_Inc_Flg3_A']]
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Prop_Inc_Flg3_A'] = data['Prop_Inc_Flg3_A'].fillna(0)

In [ ]:
#支付方式B在收入中占比
data_t_C = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_C.groupby('id', as_index=False).count()
tmp = data_t_C[data_t_C['Dat_Flg3_Cd']=='B']
tmp = tmp.groupby('id', as_index=False)['Dat_Flg3_Cd'].count()
tmp.rename(columns={'Dat_Flg3_Cd':'Num_Inc_Flg3'}, inplace=True)
data_t_sta = pd.merge(data_t_sta, tmp, on='id', how='left')
data_t_sta['Num_Inc_Flg3'] = data_t_sta['Num_Inc_Flg3'].fillna(0)
data_t_sta['Prop_Inc_Flg3_B'] = data_t_sta.apply(lambda x:x['Num_Inc_Flg3']/x['Dat_Flg3_Cd'], axis=1)
data_t_sta = data_t_sta.loc[:, ['id', 'Prop_Inc_Flg3_B']]
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Prop_Inc_Flg3_B'] = data['Prop_Inc_Flg3_B'].fillna(0)

In [ ]:
#支付方式C在收入中占比
data_t_C = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_C.groupby('id', as_index=False).count()
tmp = data_t_C[data_t_C['Dat_Flg3_Cd']=='C']
tmp = tmp.groupby('id', as_index=False)['Dat_Flg3_Cd'].count()
tmp.rename(columns={'Dat_Flg3_Cd':'Num_Inc_Flg3'}, inplace=True)
data_t_sta = pd.merge(data_t_sta, tmp, on='id', how='left')
data_t_sta['Num_Inc_Flg3'] = data_t_sta['Num_Inc_Flg3'].fillna(0)
data_t_sta['Prop_Inc_Flg3_C'] = data_t_sta.apply(lambda x:x['Num_Inc_Flg3']/x['Dat_Flg3_Cd'], axis=1)
data_t_sta = data_t_sta.loc[:, ['id', 'Prop_Inc_Flg3_C']]
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Prop_Inc_Flg3_C'] = data['Prop_Inc_Flg3_C'].fillna(0)

In [ ]:
#支付方式A在支出中占比
data_t_C = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_C.groupby('id', as_index=False).count()
tmp = data_t_C[data_t_C['Dat_Flg3_Cd']=='A']
tmp = tmp.groupby('id', as_index=False)['Dat_Flg3_Cd'].count()
tmp.rename(columns={'Dat_Flg3_Cd':'Num_Pay_Flg3'}, inplace=True)
data_t_sta = pd.merge(data_t_sta, tmp, on='id', how='left')
data_t_sta['Num_Pay_Flg3'] = data_t_sta['Num_Pay_Flg3'].fillna(0)
data_t_sta['Prop_Pay_Flg3_A'] = data_t_sta.apply(lambda x:x['Num_Pay_Flg3']/x['Dat_Flg3_Cd'], axis=1)
data_t_sta = data_t_sta.loc[:, ['id', 'Prop_Pay_Flg3_A']]
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Prop_Pay_Flg3_A'] = data['Prop_Pay_Flg3_A'].fillna(0)

In [ ]:
#支付方式B在支出中占比
data_t_C = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_C.groupby('id', as_index=False).count()
tmp = data_t_C[data_t_C['Dat_Flg3_Cd']=='B']
tmp = tmp.groupby('id', as_index=False)['Dat_Flg3_Cd'].count()
tmp.rename(columns={'Dat_Flg3_Cd':'Num_Pay_Flg3'}, inplace=True)
data_t_sta = pd.merge(data_t_sta, tmp, on='id', how='left')
data_t_sta['Num_Pay_Flg3'] = data_t_sta['Num_Pay_Flg3'].fillna(0)
data_t_sta['Prop_Pay_Flg3_B'] = data_t_sta.apply(lambda x:x['Num_Pay_Flg3']/x['Dat_Flg3_Cd'], axis=1)
data_t_sta = data_t_sta.loc[:, ['id', 'Prop_Pay_Flg3_B']]
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Prop_Pay_Flg3_B'] = data['Prop_Pay_Flg3_B'].fillna(0)

In [ ]:
#支付方式C在支出中占比
data_t_C = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_C.groupby('id', as_index=False).count()
tmp = data_t_C[data_t_C['Dat_Flg3_Cd']=='C']
tmp = tmp.groupby('id', as_index=False)['Dat_Flg3_Cd'].count()
tmp.rename(columns={'Dat_Flg3_Cd':'Num_Pay_Flg3'}, inplace=True)
data_t_sta = pd.merge(data_t_sta, tmp, on='id', how='left')
data_t_sta['Num_Pay_Flg3'] = data_t_sta['Num_Pay_Flg3'].fillna(0)
data_t_sta['Prop_Pay_Flg3_C'] = data_t_sta.apply(lambda x:x['Num_Pay_Flg3']/x['Dat_Flg3_Cd'], axis=1)
data_t_sta = data_t_sta.loc[:, ['id', 'Prop_Pay_Flg3_C']]
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Prop_Pay_Flg3_C'] = data['Prop_Pay_Flg3_C'].fillna(0)

In [ ]:
#总支出金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta.groupby('id', as_index=False)['cny_trx_amt'].sum()
data_t_sta.rename(columns={'cny_trx_amt':'Pay_Amo_B'}, inplace = True)

group_names = ['I', 'H', 'G', 'F', 'E', 'D', 'C', 'B', 'A']
data_t_sta['Pay_Amo_B'] = pd.cut(data_t_sta['Pay_Amo_B'], [-100000000, -100000, -50000, -20000, -10000, -5000, -2000, -1000, -100, 0], labels=group_names)

data = pd.merge(data, data_t_sta, on='id', how="left")
data['Pay_Amo_B'] = data['Pay_Amo_B'].fillna('A')

In [ ]:
#历史最大支出金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta.groupby('id', as_index=False)['cny_trx_amt'].min()
data_t_sta.rename(columns={'cny_trx_amt':'Max_Pay_Amo_B'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Max_Pay_Amo_B'] = data['Max_Pay_Amo_B'].fillna(0)

In [ ]:
#历史最小支出金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta.groupby('id', as_index=False)['cny_trx_amt'].max()
data_t_sta.rename(columns={'cny_trx_amt':'Min_Pay_Amo_B'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Min_Pay_Amo_B'] = data['Min_Pay_Amo_B'].fillna(0)

In [ ]:
#平均每次支出
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta.groupby('id', as_index=False)['cny_trx_amt'].mean()
data_t_sta.rename(columns={'cny_trx_amt':'Mean_Pay_Amo_B'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Mean_Pay_Amo_B'] = data['Mean_Pay_Amo_B'].fillna(0)

In [ ]:
#支出方差
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta.groupby('id', as_index=False)['cny_trx_amt'].var()
data_t_sta.rename(columns={'cny_trx_amt':'Var_Pay_Amo_B'}, inplace = True)
data_t_sta.loc[data_t_sta['Var_Pay_Amo_B']==0, 'Var_Pay_Amo_B'] = -1
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Var_Pay_Amo_B'] = data['Var_Pay_Amo_B'].fillna(-1)

In [ ]:
#支出中位数
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta.groupby('id', as_index=False)['cny_trx_amt'].median()
data_t_sta.rename(columns={'cny_trx_amt':'Med_Pay_Amo_B'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Med_Pay_Amo_B'] = data['Med_Pay_Amo_B'].fillna(0)

In [ ]:
#支出次数
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta.groupby('id', as_index=False).count()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg1_Cd']]
data_t_sta.rename(columns={'Dat_Flg1_Cd':'Pay_Cou_B'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Pay_Cou_B'] = data['Pay_Cou_B'].fillna(0)

In [ ]:
#总收入金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta.groupby('id', as_index=False)['cny_trx_amt'].sum()
data_t_sta.rename(columns={'cny_trx_amt':'Pay_Amo_C'}, inplace = True)

group_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
data_t_sta['Pay_Amo_C'] = pd.cut(data_t_sta['Pay_Amo_C'], [0, 100, 1000, 2000, 5000, 10000, 20000, 50000, 100000, 100000000], labels=group_names)

data = pd.merge(data, data_t_sta, on='id', how="left")
data['Pay_Amo_C'] = data['Pay_Amo_C'].fillna('A')

In [ ]:
#历史最大收入金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta.groupby('id', as_index=False)['cny_trx_amt'].max()
data_t_sta.rename(columns={'cny_trx_amt':'Max_Pay_Amo_C'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Max_Pay_Amo_C'] = data['Max_Pay_Amo_C'].fillna(0)

In [ ]:
#历史最小收入金额
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta.groupby('id', as_index=False)['cny_trx_amt'].min()
data_t_sta.rename(columns={'cny_trx_amt':'Min_Pay_Amo_C'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Min_Pay_Amo_C'] = data['Min_Pay_Amo_C'].fillna(0)

In [ ]:
#平均每次收入
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta.groupby('id', as_index=False)['cny_trx_amt'].mean()
data_t_sta.rename(columns={'cny_trx_amt':'Mean_Pay_Amo_C'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Mean_Pay_Amo_C'] = data['Mean_Pay_Amo_C'].fillna(0)

In [ ]:
#收入方差
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta.groupby('id', as_index=False)['cny_trx_amt'].var()
data_t_sta.rename(columns={'cny_trx_amt':'Var_Pay_Amo_C'}, inplace = True)
data_t_sta.loc[data_t_sta['Var_Pay_Amo_C']==0, 'Var_Pay_Amo_C'] = -1
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Var_Pay_Amo_C'] = data['Var_Pay_Amo_C'].fillna(-1)

In [ ]:
#收入中位数
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta.groupby('id', as_index=False)['cny_trx_amt'].median()
data_t_sta.rename(columns={'cny_trx_amt':'Med_Pay_Amo_C'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Med_Pay_Amo_C'] = data['Med_Pay_Amo_C'].fillna(0)

In [ ]:
#收入次数
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta.groupby('id', as_index=False).count()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg1_Cd']]
data_t_sta.rename(columns={'Dat_Flg1_Cd':'Pay_Cou_C'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Pay_Cou_C'] = data['Pay_Cou_C'].fillna(0)

In [ ]:
#剩余资产
data_t_sta = data_t.groupby('id', as_index=False)['cny_trx_amt'].sum()
data_t_sta.rename(columns={'cny_trx_amt':'Res_Aset'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Res_Aset'] = data['Res_Aset'].fillna(0)

In [ ]:
#收入最高分类等级
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta.groupby('id', as_index=False)['Trx_Cod2_Cd'].max()
data_t_sta.rename(columns={'Trx_Cod2_Cd':'Max_Trx_Cod2_Cd_C'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Max_Trx_Cod2_Cd_C'] = data['Max_Trx_Cod2_Cd_C'].fillna('-1')

In [ ]:
#收入最低分类等级
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta.groupby('id', as_index=False)['Trx_Cod2_Cd'].min()
data_t_sta.rename(columns={'Trx_Cod2_Cd':'Min_Trx_Cod2_Cd_C'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Min_Trx_Cod2_Cd_C'] = data['Min_Trx_Cod2_Cd_C'].fillna('-1')

In [ ]:
#支出最高分类等级 
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta.groupby('id', as_index=False)['Trx_Cod2_Cd'].max()
data_t_sta.rename(columns={'Trx_Cod2_Cd':'Max_Trx_Cod2_Cd_B'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Max_Trx_Cod2_Cd_B'] = data['Max_Trx_Cod2_Cd_B'].fillna('-1')

In [ ]:
#支出最低分类等级 
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta.groupby('id', as_index=False)['Trx_Cod2_Cd'].min()
data_t_sta.rename(columns={'Trx_Cod2_Cd':'Min_Trx_Cod2_Cd_B'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Min_Trx_Cod2_Cd_B'] = data['Min_Trx_Cod2_Cd_B'].fillna('-1')

In [ ]:
#最常见收入分类等级 
data_t_sta = data_t[data_t['Dat_Flg1_Cd'] == 'C']
data_t_sta = data_t.groupby(['id', 'Trx_Cod1_Cd'], as_index=False).count()
data_t_sta = data_t_sta.sort_values('flag', ascending=False)
data_t_sta = data_t_sta.groupby('id').first().reset_index()
data_t_sta = data_t_sta.loc[:, ['id', 'Trx_Cod1_Cd']]
data_t_sta.rename(columns={'Trx_Cod1_Cd':'Fre_Trx_Cod1_Cd_C'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Fre_Trx_Cod1_Cd_C'] = data['Fre_Trx_Cod1_Cd_C'].fillna('-1')

In [ ]:
#最常见支出分类等级 
data_t_sta = data_t[data_t['Dat_Flg1_Cd'] == 'B']
data_t_sta = data_t.groupby(['id', 'Trx_Cod1_Cd'], as_index=False).count()
data_t_sta = data_t_sta.sort_values('flag', ascending=False)
data_t_sta = data_t_sta.groupby('id').first().reset_index()
data_t_sta = data_t_sta.loc[:, ['id', 'Trx_Cod1_Cd']]
data_t_sta.rename(columns={'Trx_Cod1_Cd':'Fre_Trx_Cod1_Cd_B'}, inplace = True)
data = pd.merge(data, data_t_sta, on='id', how="left")
data['Fre_Trx_Cod1_Cd_B'] = data['Fre_Trx_Cod1_Cd_B'].fillna('-1')

In [ ]:
# 活跃时间
def function(a,b):
    return a-b

data_t.sort_values('trx_tm', ascending=True, inplace=True)
firstTime = data_t.groupby('id', as_index=False).first()
firstTime['ddl'] = pd.to_datetime('2019-07-01 00:00:00', format='%Y-%m-%d %H:%M:%S')
firstTime['Tim_Dif'] = firstTime.apply(lambda x:function(x['ddl'],x['trx_tm']),axis=1)
timedif  = firstTime.loc[:, ['id', 'Tim_Dif']]
timedif = timedif.fillna(0)
timedif['Tim_Dif'] = timedif['Tim_Dif'].astype('timedelta64[s]').astype(int)
data = pd.merge(data, timedif, on='id', how="left")
data['Tim_Dif'] = data['Tim_Dif'].fillna(0)

In [ ]:
# 有支出日期数量
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t.groupby(['id', 'trx_day'], as_index=False).count()
data_t_sta = data_t_sta.groupby('id', as_index=False).count()
data_t_sta.rename(columns={'trx_day': 'Num_Tran_Day_B'}, inplace=True)
data_t_sta = data_t_sta.loc[:, ['id', 'Num_Tran_Day_B']]
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Num_Tran_Day_B'] = data['Num_Tran_Day_B'].fillna(0)

In [ ]:
# 有收入日期数量
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t.groupby(['id', 'trx_day'], as_index=False).count()
data_t_sta = data_t_sta.groupby('id', as_index=False).count()
data_t_sta.rename(columns={'trx_day': 'Num_Tran_Day_C'}, inplace=True)
data_t_sta = data_t_sta.loc[:, ['id', 'Num_Tran_Day_C']]
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Num_Tran_Day_C'] = data['Num_Tran_Day_C'].fillna(0)

In [ ]:
#平均收入间隔时间
def function(a,b,c):
    return (a-b)/c

data_t_C = data_t[data_t['Dat_Flg1_Cd'] == 'C']

data_t_C.sort_values('trx_tm', ascending=True, inplace=True)
firstTime = data_t_C.groupby('id', as_index=False).first()

data_t_C.sort_values('trx_tm', ascending=False, inplace=True)
lastTime = data_t_C.groupby('id', as_index=False).first()
lastTime = lastTime.loc[:,['id', 'trx_tm']]
lastTime.rename(columns={'trx_tm':'Las_Tim'}, inplace = True)

firstTime = pd.merge(firstTime, lastTime, on='id', how='left')

data_t_C['count'] = 1
Count = data_t_C.groupby('id', as_index=False)['count'].count()
firstTime = pd.merge(firstTime, Count, on='id', how='left')

firstTime['Ave_Inc_Int'] = firstTime.apply(lambda x:function(x['Las_Tim'], x['trx_tm'], x['count']),axis=1)
aveInt = firstTime.loc[:,['id', 'Ave_Inc_Int']]
aveInt['Ave_Inc_Int'] = aveInt['Ave_Inc_Int'].astype('timedelta64[h]').astype(int)
aveInt.loc[aveInt['Ave_Inc_Int']<=0, 'Ave_Inc_Int'] = -1
data = pd.merge(data, aveInt, on='id', how='left')
data['Ave_Inc_Int']=data['Ave_Inc_Int'].fillna(-1)

In [ ]:
#平均支出间隔时间
def function(a,b,c):
    return (a-b)/c

data_t_C = data_t[data_t['Dat_Flg1_Cd'] == 'B']
data_t_C.sort_values('trx_tm', ascending=True, inplace=True)
firstTime = data_t_C.groupby('id', as_index=False).first()
data_t_C.sort_values('trx_tm', ascending=False, inplace=True)
lastTime = data_t_C.groupby('id', as_index=False).first()
lastTime = lastTime.loc[:,['id', 'trx_tm']]
lastTime.rename(columns={'trx_tm':'Las_Tim'}, inplace = True)
firstTime = pd.merge(firstTime, lastTime, on='id', how='left')
data_t_C['count'] = 1
Count = data_t_C.groupby('id', as_index=False)['count'].count()
firstTime = pd.merge(firstTime, Count, on='id', how='left')
firstTime['Ave_Pay_Int'] = firstTime.apply(lambda x:function(x['Las_Tim'], x['trx_tm'], x['count']),axis=1)
aveInt = firstTime.loc[:,['id', 'Ave_Pay_Int']]
aveInt['Ave_Pay_Int'] = aveInt['Ave_Pay_Int'].astype('timedelta64[h]').astype(int)
aveInt.loc[aveInt['Ave_Pay_Int']<=0, 'Ave_Pay_Int'] = -1
data = pd.merge(data, aveInt, on='id', how='left')
data['Ave_Pay_Int']=data['Ave_Pay_Int'].fillna(-1)

In [ ]:
#用户最后一次支出时间/金额/一级代码/二级代码/交易方式
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta.sort_values('trx_tm', ascending=False, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'trx_tm', 'cny_trx_amt', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'Dat_Flg3_Cd']]
data_t_sta.rename(columns={'trx_tm':'Last_Pay_Time', 'cny_trx_amt':'Last_Pay_Amt', 'Trx_Cod1_Cd':'Last_Pay_Cod1',
                           'Trx_Cod2_Cd':'Last_Pay_Cod2', 'Dat_Flg3_Cd':'Last_Pay_Flg3'}, inplace=True)
data_t_sta['Last_Pay_Time'] = pd.to_numeric(data_t_sta['Last_Pay_Time'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Last_Pay_Time'] = data['Last_Pay_Time'].fillna(-1)
data['Last_Pay_Amt'] = data['Last_Pay_Amt'].fillna(0)
data['Last_Pay_Cod1'] = data['Last_Pay_Cod1'].fillna('-1')
data['Last_Pay_Cod2'] = data['Last_Pay_Cod2'].fillna('-1')
data['Last_Pay_Flg3'] = data['Last_Pay_Flg3'].fillna('-1')

In [ ]:
#用户最后一次收入时间/金额/一级代码/二级代码/交易方式
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta.sort_values('trx_tm', ascending=False, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'trx_tm', 'cny_trx_amt', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'Dat_Flg3_Cd']]
data_t_sta.rename(columns={'trx_tm':'Last_Inc_Time', 'cny_trx_amt':'Last_Inc_Amt', 'Trx_Cod1_Cd':'Last_Inc_Cod1',
                           'Trx_Cod2_Cd':'Last_Inc_Cod2', 'Dat_Flg3_Cd':'Last_Inc_Flg3'}, inplace=True)
data_t_sta['Last_Inc_Time'] = pd.to_numeric(data_t_sta['Last_Inc_Time'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Last_Inc_Time'] = data['Last_Inc_Time'].fillna(-1)
data['Last_Inc_Amt'] = data['Last_Inc_Amt'].fillna(0)
data['Last_Inc_Cod1'] = data['Last_Inc_Cod1'].fillna('-1')
data['Last_Inc_Cod2'] = data['Last_Inc_Cod2'].fillna('-1')
data['Last_Inc_Flg3'] = data['Last_Inc_Flg3'].fillna('-1')

In [ ]:
#用户第一次支出时间/金额/一级代码/二级代码/交易方式
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta.sort_values('trx_tm', ascending=True, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'trx_tm', 'cny_trx_amt', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'Dat_Flg3_Cd']]
data_t_sta.rename(columns={'trx_tm':'First_Pay_Time', 'cny_trx_amt':'First_Pay_Amt', 'Trx_Cod1_Cd':'First_Pay_Cod1', 
                           'Trx_Cod2_Cd':'First_Pay_Cod2', 'Dat_Flg3_Cd':'First_Pay_Flg3'}, inplace=True)
data_t_sta['First_Pay_Time'] = pd.to_numeric(data_t_sta['First_Pay_Time'])
data = pd.merge(data, data_t_sta, on='id', how='left')
data['First_Pay_Time'] = data['First_Pay_Time'].fillna(-1)
data['First_Pay_Amt'] = data['First_Pay_Amt'].fillna(0)
data['First_Pay_Cod1'] = data['First_Pay_Cod1'].fillna('-1')
data['First_Pay_Cod2'] = data['First_Pay_Cod2'].fillna('-1')
data['First_Pay_Flg3'] = data['First_Pay_Flg3'].fillna('-1')

In [ ]:
#用户第一次收入时间/金额/一级代码/二级代码/交易方式
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta.sort_values('trx_tm', ascending=True, inplace=True)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'trx_tm', 'cny_trx_amt', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd', 'Dat_Flg3_Cd']]
data_t_sta.rename(columns={'trx_tm':'First_Inc_Time', 'cny_trx_amt':'First_Inc_Amt', 'Trx_Cod1_Cd':'First_Inc_Cod1',
                           'Trx_Cod2_Cd':'First_Inc_Cod2', 'Dat_Flg3_Cd':'First_Inc_Flg3'}, inplace=True)
data_t_sta['First_Inc_Time'] = pd.to_numeric(data_t_sta['First_Inc_Time'])

group_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
data_t_sta['First_Inc_Amt'] = pd.cut(data_t_sta['First_Inc_Amt'], [0, 10, 100, 200, 500, 1000, 2000, 5000, 10000, 100000000], labels=group_names)

data = pd.merge(data, data_t_sta, on='id', how='left')
data['First_Inc_Time'] = data['First_Inc_Time'].fillna(-1)
data['First_Inc_Amt'] = data['First_Inc_Amt'].fillna('A')
data['First_Inc_Cod1'] = data['First_Inc_Cod1'].fillna('-1')
data['First_Inc_Cod2'] = data['First_Inc_Cod2'].fillna('-1')
data['First_Inc_Flg3'] = data['First_Inc_Flg3'].fillna('-1')

In [ ]:
# 用户最后一次交易收支方向
data_t.sort_values('trx_tm', ascending=False, inplace=True)
data_t_sta = data_t.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg1_Cd']]
data_t_sta.rename(columns={'Dat_Flg1_Cd':'Last_Trans_Flg1'}, inplace=True)
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Last_Trans_Flg1'] = data['Last_Trans_Flg1'].fillna('-1')

In [ ]:
# 用户第一次交易收支方向
data_t.sort_values('trx_tm', ascending=True, inplace=True)
data_t_sta = data_t.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'Dat_Flg1_Cd']]
data_t_sta.rename(columns={'Dat_Flg1_Cd':'First_Trans_Flg1'}, inplace=True)
data = pd.merge(data, data_t_sta, on='id', how='left')
data['First_Trans_Flg1'] = data['First_Trans_Flg1'].fillna('-1')

In [ ]:
#支出最常用时间段
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='B']
data_t_sta = data_t_sta.groupby(['id', 'trx_hour'], as_index=False).count()
data_t_sta = data_t_sta.sort_values('flag', ascending=False)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'trx_hour']]
data_t_sta.rename(columns={'trx_hour':'Mos_Pay_Hour'}, inplace=True)
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Mos_Pay_Hour'] = data['Mos_Pay_Hour'].fillna('-1')

In [ ]:
#收入最常用时间段
data_t_sta = data_t[data_t['Dat_Flg1_Cd']=='C']
data_t_sta = data_t_sta.groupby(['id', 'trx_hour'], as_index=False).count()
data_t_sta = data_t_sta.sort_values('flag', ascending=False)
data_t_sta = data_t_sta.groupby('id', as_index=False).first()
data_t_sta = data_t_sta.loc[:, ['id', 'trx_hour']]
data_t_sta.rename(columns={'trx_hour':'Mos_Inc_Hour'}, inplace=True)
data = pd.merge(data, data_t_sta, on='id', how='left')
data['Mos_Inc_Hour'] = data['Mos_Inc_Hour'].fillna('-1')

### APP表

In [ ]:
# app访问次数
data_b_sta = data_b.groupby('id', as_index=False).count()
data_b_sta = data_b_sta.loc[:, ['id', 'page_no']]
data_b_sta.rename(columns={'page_no':'Vis_cou'}, inplace = True)
data = pd.merge(data, data_b_sta, on='id', how='left')
data['Vis_cou'] = data['Vis_cou'].fillna(0)

In [ ]:
#最常访问页面
data_b_sta = data_b.groupby(['id', 'page_no'], as_index=False).count()
data_b_sta = data_b_sta.sort_values('flag', ascending=False)
data_b_sta = data_b_sta.groupby('id').first().reset_index()
data_b_sta = data_b_sta.loc[:, ['id', 'page_no']]
data_b_sta.rename(columns={'page_no':'Mos_Vis_Pag'}, inplace=True)
data = pd.merge(data, data_b_sta, on='id', how='left')
data['Mos_Vis_Pag'] = data['Mos_Vis_Pag'].fillna('-1')

In [ ]:
#访问页面数量
data_b_sta = data_b.groupby(['id', 'page_no'], as_index=False).count()
data_b_sta = data_b_sta.groupby('id', as_index=False).count()
data_b_sta = data_b_sta.loc[:, ['id','page_no']]
data_b_sta.rename(columns={'page_no':'Num_Pag_Vis'}, inplace=True)
data = pd.merge(data, data_b_sta, on='id', how='left')
data['Num_Pag_Vis'] = data['Num_Pag_Vis'].fillna(0)

In [ ]:
#平均访问间隔
def function(a,b,c):
    return (a-b)/c

data_b.sort_values('page_tm', ascending=True, inplace=True)
firstTime = data_b.groupby('id', as_index=False).first()
firstTime = firstTime.loc[:, ['id', 'page_tm']]
firstTime.rename(columns={'page_tm':'Fir_Tim'}, inplace=True)
data_b.sort_values('page_tm', ascending=False, inplace=True)
lastTime = data_b.groupby('id', as_index=False).first()
lastTime = lastTime.loc[:, ['id', 'page_tm']]
lastTime.rename(columns={'page_tm':'Las_Tim'}, inplace=True)

data_b_sta = data_b.groupby('id', as_index=False).count()
data_b_sta = pd.merge(data_b_sta, firstTime, on='id', how='left')
data_b_sta = pd.merge(data_b_sta, lastTime, on='id', how='left')
data_b_sta['Ave_Vis_Int'] = data_b_sta.apply(lambda x:function(x['Las_Tim'], x['Fir_Tim'], x['flag']), axis=1)
data_b_sta = data_b_sta.loc[:, ['id', 'Ave_Vis_Int']]
data_b_sta['Ave_Vis_Int'] = data_b_sta['Ave_Vis_Int'].astype('timedelta64[h]').astype(int)
data_b_sta.loc[data_b_sta['Ave_Vis_Int']<=0, 'Ave_Vis_Int'] = -1
data = pd.merge(data, data_b_sta, on='id', how='left')
data['Ave_Vis_Int'] = data['Ave_Vis_Int'].fillna(-1)

In [ ]:
#常访问app时段
data_b_sta = data_b.groupby(['id', 'page_hour'], as_index=False).count()
data_b_sta = data_b_sta.sort_values('flag', ascending=False)
data_b_sta = data_b_sta.groupby('id').first().reset_index()
data_b_sta = data_b_sta.loc[:, ['id', 'page_hour']]
data_b_sta.rename(columns={'page_hour':'Mos_Vis_Hour'}, inplace=True)
data = pd.merge(data, data_b_sta, on='id', how='left')
data['Mos_Vis_Hour'] = data['Mos_Vis_Hour'].fillna('-1')

In [ ]:
#单日最高访问量
data_b_sta = data_b.groupby(['id', 'page_day'], as_index=False).count()
data_b_sta = data_b_sta.groupby('id', as_index=False)['flag'].max()
data_b_sta.rename(columns={'flag':'Max_Vis_Num'}, inplace=True)
data = pd.merge(data, data_b_sta, on='id', how='left')
data['Max_Vis_Num'] = data['Max_Vis_Num'].fillna(0)                                             

In [ ]:
#平均日访问量
data_b_sta = data_b.groupby('id', as_index=False).count()
tmp = data_b.groupby(['id', 'page_day'], as_index=False).count()
tmp = tmp.groupby('id', as_index=False)['page_day'].count()
tmp.rename(columns={'page_day':'num_day'}, inplace = True)
data_b_sta = pd.merge(data_b_sta, tmp, on='id', how='left')
data_b_sta['Ave_Vis_Num'] = data_b_sta.apply(lambda x:x['flag']/x['num_day'], axis=1)
data_b_sta = data_b_sta.loc[:, ['id', 'Ave_Vis_Num']]
data = pd.merge(data, data_b_sta, on='id', how='left')
data['Ave_Vis_Num'] = data['Ave_Vis_Num'].fillna(0)  

In [ ]:
#用户最后一次访问APP的页面/时间
data_b.sort_values('page_tm', ascending=False, inplace=True)
data_b_sta = data_b.groupby('id', as_index=False).first()
data_b_sta = data_b_sta.loc[:, ['id', 'page_no', 'page_tm']]
data_b_sta.rename(columns={'page_no':'Last_Vis_no', 'page_tm':'Last_Vis_Time'}, inplace=True)
data_b_sta['Last_Vis_Time'] = pd.to_numeric(data_b_sta['Last_Vis_Time'])
data = pd.merge(data, data_b_sta, on='id', how='left')
data['Last_Vis_no'] = data['Last_Vis_no'].fillna('-1')
data['Last_Vis_Time'] = data['Last_Vis_Time'].fillna(-1)

In [ ]:
#用户第一次访问APP的页面/时间
data_b.sort_values('page_tm', ascending=True, inplace=True)
data_b_sta = data_b.groupby('id', as_index=False).first()
data_b_sta = data_b_sta.loc[:, ['id', 'page_no', 'page_tm']]
data_b_sta.rename(columns={'page_no':'First_Vis_no', 'page_tm':'First_Vis_Time'}, inplace=True)
data_b_sta['First_Vis_Time'] = pd.to_numeric(data_b_sta['First_Vis_Time'])
data = pd.merge(data, data_b_sta, on='id', how='left')
data['First_Vis_no'] = data['First_Vis_no'].fillna('-1')
data['First_Vis_Time'] = data['First_Vis_Time'].fillna(-1)

In [ ]:
normal_feature = ['cur_debit_cnt', 'cur_credit_cnt', 'cur_debit_min_opn_dt_cnt', 'cur_credit_min_opn_dt_cnt', 'age',
                  'job_year', 'frs_agn_dt_cnt', 'Max_Pay_Amo_B', 'Min_Pay_Amo_B', 'Mean_Pay_Amo_B',
                  'Var_Pay_Amo_B', 'Pay_Cou_B', 'Med_Pay_Amo_B', 'Max_Pay_Amo_C', 'Min_Pay_Amo_C',
                  'Mean_Pay_Amo_C', 'Var_Pay_Amo_C', 'Pay_Cou_C', 'Med_Pay_Amo_C', 'Num_Tran_Day_B',
                  'Num_Tran_Day_C',  'Res_Aset', 'Tim_Dif', 'Ave_Inc_Int', 'Ave_Pay_Int',
                  
                  'Max_Trx_Cod2_Cd_B', 'Max_Trx_Cod2_Cd_C','Min_Trx_Cod2_Cd_B', 'Vis_cou', 'Max_Vis_Num',
                  'Min_Trx_Cod2_Cd_C', 'Num_Pag_Vis','Ave_Vis_Int',  'Ave_Vis_Num', 'Last_Pay_Time',
                  'Last_Pay_Amt', 'Last_Pay_Cod2', 'First_Pay_Time', 'First_Pay_Amt', 'First_Pay_Cod2',
                  'Last_Inc_Time','Last_Inc_Amt','Last_Inc_Cod2', 'First_Inc_Time', 'First_Inc_Cod2',
                  'Last_Vis_Time', 'First_Vis_Time', 'Num_Trans_Flg3_A', 'Num_Trans_Flg3_B', 'Num_Trans_Flg3_C',
                  
                  'Num_Inc_Flg3_A', 'Num_Inc_Flg3_B', 'Num_Inc_Flg3_C', 'Num_Pay_Flg3_A', 'Num_Pay_Flg3_B',
                  'Num_Pay_Flg3_C', 'First_Trans_Cod2_A', 'First_Trans_Time_A', 'First_Trans_Amt_A', 'First_Trans_Cod2_B',
                  'First_Trans_Time_B', 'First_Trans_Amt_B', 'First_Trans_Cod2_C', 'First_Trans_Time_C', 'First_Trans_Amt_C',
                  'First_Inc_Cod2_A', 'First_Inc_Time_A', 'First_Inc_Amt_A', 'First_Inc_Cod2_B', 'First_Inc_Time_B',
                  'First_Inc_Amt_B', 'First_Inc_Cod2_C', 'First_Inc_Time_C', 'First_Inc_Amt_C', 'First_Pay_Cod2_A',
                  
                  'First_Pay_Time_A', 'First_Pay_Amt_A', 'First_Pay_Cod2_B', 'First_Pay_Time_B', 'First_Pay_Amt_B',
                  'First_Pay_Cod2_C', 'First_Pay_Time_C', 'First_Pay_Amt_C', 'Last_Trans_Cod2_A', 'Last_Trans_Time_A',
                  'Last_Trans_Amt_A', 'Last_Trans_Cod2_B', 'Last_Trans_Time_B', 'Last_Trans_Amt_B', 'Last_Trans_Cod2_C',
                  'Last_Trans_Time_C', 'Last_Trans_Amt_C', 'Last_Inc_Cod2_A', 'Last_Inc_Time_A', 'Last_Inc_Amt_A',
                  'Last_Inc_Cod2_B', 'Last_Inc_Time_B', 'Last_Inc_Amt_B', 'Last_Inc_Cod2_C', 'Last_Inc_Time_C',
                  
                  'Last_Inc_Amt_C', 'Last_Pay_Cod2_A', 'Last_Pay_Time_A', 'Last_Pay_Amt_A', 'Last_Pay_Cod2_B',
                  'Last_Pay_Time_B', 'Last_Pay_Amt_B', 'Last_Pay_Cod2_C', 'Last_Pay_Time_C', 'Last_Pay_Amt_C',
                  'Prop_Trans_Flg3_A', 'Prop_Trans_Flg3_B', 'Prop_Trans_Flg3_C', 'Prop_Inc_Flg3_A', 'Prop_Inc_Flg3_B',
                  'Prop_Inc_Flg3_C', 'Prop_Pay_Flg3_A', 'Prop_Pay_Flg3_B', 'Prop_Pay_Flg3_C'] 

one_hot_feature = ['cur_debit_crd_lvl', 'hld_crd_card_grd_cd', 'crd_card_act_ind', 'l1y_crd_card_csm_amt_dlm_cd', 'atdd_type', 
                   'perm_crd_lmt_cd', 'gdr_cd', 'mrg_situ_cd', 'ic_ind', 'fr_or_sh_ind', 
                   'dnl_mbl_bnk_ind', 'dnl_bind_cmb_lif_ind', 'hav_car_grp_ind', 'hav_hou_grp_ind', 'l6mon_agn_ind', 
                   'vld_rsk_ases_ind', 'fin_rsk_ases_grd_cd', 'confirm_rsk_ases_lvl_typ_cd', 'cust_inv_rsk_endu_lvl_cd', 'l6mon_daim_aum_cd', 
                   'tot_ast_lvl_cd', 'pot_ast_lvl_cd', 'bk1_cur_year_mon_avg_agn_amt_cd', 'loan_act_ind', 'pl_crd_lmt_cd', 
                   'Fre_Trans_Flg3', 'Fre_Trx_Cod1_Cd_B', 'Fre_Trx_Cod1_Cd_C', 'Mos_Vis_Pag', 'Mos_Vis_Hour', 
                   'Pay_Amo_B', 'Pay_Amo_C', 'Last_Pay_Cod1', 'Last_Pay_Flg3', 'First_Pay_Cod1',
                   'First_Pay_Flg3', 'Last_Inc_Cod1', 'Last_Inc_Flg3', 'First_Inc_Cod1', 'First_Inc_Flg3',
                   'Last_Trans_Flg1', 'First_Trans_Flg1', 'Last_Vis_no', 'First_Vis_no', 'First_Inc_Amt', #'Last_Inc_Amt',#!!!
                  'Fre_Inc_Flg3', 'Fre_Pay_Flg3', 'Mos_Inc_Hour', 'Mos_Pay_Hour', 'First_Trans_Cod1_A',
                   'First_Trans_Flg1_A', 'First_Trans_Cod1_B', 'First_Trans_Flg1_B', 'First_Trans_Cod1_C', 'First_Trans_Flg1_C',
                   'First_Inc_Cod1_A', 'First_Inc_Cod1_B', 'First_Inc_Cod1_C', 'First_Pay_Cod1_A', 'First_Pay_Cod1_B',
                   'First_Pay_Cod1_C', 'Last_Trans_Cod1_A', 'Last_Trans_Flg1_A', 'Last_Trans_Cod1_B', 'Last_Trans_Flg1_B',
                   'Last_Trans_Cod1_C', 'Last_Trans_Flg1_C', 'Last_Inc_Cod1_A', 'Last_Inc_Cod1_B', 'Last_Inc_Cod1_C',
                   'Last_Pay_Cod1_A', 'Last_Pay_Cod1_B', 'Last_Pay_Cod1_C']

In [ ]:
new_append = data.iloc[0, :]
new_append['id'] = '10000'
new_append['flag'] = -2
data = data.append(new_append).reset_index()

In [ ]:
data.loc[(data['cur_credit_cnt']==0), 'hld_crd_card_grd_cd'] = '0'
data.loc[(data['hld_crd_card_grd_cd']=='\N'), 'hld_crd_card_grd_cd'] = '-1'

In [ ]:
data.loc[(data['flag']==-2), 'Mos_Vis_Pag'] = 'FLS', 
data['fin_rsk_ases_grd_cd'] = data['fin_rsk_ases_grd_cd'].apply(lambda x:'-1' if x=='10' else x)

In [ ]:
for feature in one_hot_feature:
    print feature
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])

In [ ]:
for feature in normal_feature:
    data[feature] = data[feature].apply(lambda x:'-1' if x=='\N' else x)

In [ ]:
data = data.drop('index', axis=1)

In [ ]:
train = data[data.flag!=-1]
train = train[train.flag!=-2]
train_y = train.pop('flag')

train, val, train_y, val_y = train_test_split(train, train_y, test_size=0.2, random_state=2020)

test = data[data.flag==-1]
res = test[['id']]
test = test.drop('flag',axis=1)
enc = OneHotEncoder()

In [ ]:
train_x = train[normal_feature]
val_x = val[normal_feature]
test_x = test[normal_feature]

for feature in one_hot_feature:
    print feature
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a = enc.transform(train[feature].values.reshape(-1, 1))
    val_a = enc.transform(val[feature].values.reshape(-1, 1))
    test_a = enc.transform(test[feature].values.reshape(-1, 1))
    train_x = sparse.hstack((train_x.astype(float), train_a))
    val_x = sparse.hstack((val_x.astype(float), val_a))
    test_x = sparse.hstack((test_x.astype(float), test_a))
print('one-hot prepared !')

In [ ]:
print("LGB test")
clf = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=10, reg_alpha=2, reg_lambda=2,
    max_depth=-1, n_estimators=1500, objective='binary',
    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.05, min_child_weight=20, random_state=2020, n_jobs=100
)
clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], eval_metric='auc',early_stopping_rounds=50)

In [ ]:
feature_importance = clf.feature_importances_
fea = pd.DataFrame()
s_list = [i for i in range(len(feature_importance))]
fea['select_list'] = s_list
fea['value'] = feature_importance

fea1 = fea.iloc[:119,:]
print fea1.loc[fea1['value']>0]['select_list'].count() #171

fea = fea[fea['value']==0]
s_list = list(fea['select_list'])
print len(s_list)

In [ ]:
train_rs = train_x.toarray()
val_rs = val_x.toarray()
test_rs = test_x.toarray()
print train_rs.shape
print val_rs.shape
print test_rs.shape
for i in range(len(s_list)-1, -1, -1):
    train_rs = np.delete(train_rs, s_list[i], axis = 1)
    val_rs = np.delete(val_rs, s_list[i], axis = 1)
    test_rs = np.delete(test_rs, s_list[i], axis = 1)
print train_rs.shape
print val_rs.shape
print test_rs.shape

In [ ]:
print("LGB test")
clf_rs = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=10, reg_alpha=2, reg_lambda=2,
    max_depth=-1, n_estimators=1500, objective='binary',
    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.05, min_child_weight=20, random_state=2020, n_jobs=100
)
clf_rs.fit(train_rs, train_y, eval_set=[(val_rs, val_y)], eval_metric='auc',early_stopping_rounds=50)

In [ ]:
feature_importance = clf_rs.feature_importances_
print len(feature_importance)

fea = pd.DataFrame()
fea['feature_index'] = [i for i in range(110)]
fea['value'] = feature_importance[:110]
new_feature = fea.loc[fea['value']<=30]
new_feature = list(new_feature['feature_index'])
print len(new_feature)

In [ ]:
train_imp = train_rs[:,:110]
val_imp = val_rs[:,:110]
test_imp = test_rs[:,:110]
print train_imp.shape
print val_imp.shape
print test_imp.shape
for i in range(len(new_feature)-1, -1, -1):
    train_imp = np.delete(train_imp, new_feature[i], axis = 1)
    val_imp = np.delete(val_imp, new_feature[i], axis = 1)
    test_imp = np.delete(test_imp, new_feature[i], axis = 1)
print train_imp.shape
print val_imp.shape
print test_imp.shape

In [ ]:
poly1=PolynomialFeatures()
cross1=poly1.fit_transform(train_imp)
cross2=poly1.fit_transform(val_imp)
cross3=poly1.fit_transform(test_imp)
print (cross1.shape)
print (cross2.shape)
print (cross3.shape)

In [ ]:
train_cro = np.concatenate((train_rs, cross1),axis=1)
val_cro = np.concatenate((val_rs, cross2), axis=1)
test_cro = np.concatenate((test_rs, cross3),axis =1)

In [ ]:
print("LGB test")
clf_cro = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=10, reg_alpha=2, reg_lambda=2,
    max_depth=-1, n_estimators=1500, objective='binary',
    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.05, min_child_weight=20, random_state=2020, n_jobs=100
)
clf_cro.fit(train_cro, train_y, eval_set=[(val_cro, val_y)], eval_metric='auc',early_stopping_rounds=50)

In [ ]:
feature_importance = clf_cro.feature_importances_
print len(feature_importance)
fea = pd.DataFrame()
s_list = [i for i in range(len(feature_importance))]
fea['select_list'] = s_list
fea['value'] = feature_importance
fea = fea.loc[fea['value']==0]
s_list = list(fea['select_list'])
print len(s_list)

In [ ]:
train_last = train_cro
val_last = val_cro
test_last = test_cro
print train_last.shape
print val_last.shape
print test_last.shape
for i in range(len(s_list)-1, -1, -1):
    train_last = np.delete(train_last, s_list[i], axis = 1)
    val_last = np.delete(val_last, s_list[i], axis = 1)
    test_last = np.delete(test_last, s_list[i], axis = 1)
print train_last.shape
print val_last.shape
print test_last.shape

In [ ]:
print("LGB test")
clf_last = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=10, reg_alpha=2, reg_lambda=2,
    max_depth=-1, n_estimators=1500, objective='binary',
    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.05, min_child_weight=20, random_state=2020, n_jobs=100
)
clf_last.fit(train_last, train_y, eval_set=[(val_last, val_y)], eval_metric='auc',early_stopping_rounds=50)

In [ ]:
test_predict = clf.predict(test_x)
test_proba = clf.predict_proba(test_x)[:,1]
res['flag'] = test_proba
res['predict'] = test_predict

In [ ]:
res[['id', 'flag']].to_csv(path_or_buf="lgb.txt" ,sep = '\t', index=False, header=None) # 写入csv文件